<b>Write a Python Script that captures images from your webcam video stream</b><br>
<b>Extracts all Faces from the image frame (using haarcascades)</b><br>
<b>Stores the Face information into numpy arrays</b><br>

<ol>
    <li><b>Read and show video stream, capture images</b></li>
    <li><b>Detect Faces and show bounding box (haarcascade)</b></li>
    <li><b>Flatten the largest face image(gray scale) and save in a numpy array</b></li>
    <li><b>Repeat the above for multiple people to generate training data</b></li>
</ol>

In [1]:
import cv2
import numpy as np

#Init camera
cap = cv2.VideoCapture(0)

#Face Detection
face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_alt.xml")

skip = 0
face_data = [] #The list that stores all the face data
dataset_path = './Face_Data_Store/'
file_name = input("Enter name of the person: ")

while True:
    ret,frame = cap.read()
    
    if ret == False:
        continue
    
    
    faces = face_cascade.detectMultiScale(frame,1.3,5)
    #print(faces) #This stores all the faces inside a list, note that a face consists of 4 components, (x,y,w,h)
    if len(faces)==0:
        continue
    
    #Now we need to store the largest face image, note that the area of a rectangle is height * breadth
    #So in this case area = w * h i.e face[2] * face[3] and we need to sort according to this value
    faces = sorted(faces, key = lambda f: f[2] * f[3])
    
    #Now let's iterate over the faces to put a rectangle alongside the face 
    
    face_section = []
    
    for face in faces[-1:]: #Taking the last value as it is the largest value
        x,y,w,h = face
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,255),2)
        
        #Extract (Crop out the required face): Region of interest
        offset = 10
        face_section = frame[y-offset:y+h+offset, x-offset:x+w+offset] #Taking 10 pixels more form each side
        face_section = cv2.resize(face_section, (100,100))
        
        skip += 1
        if skip%10 == 0: #Capturing every 10th image
            face_data.append(face_section)
            print(len(face_data))
        
    face_section = np.asarray(face_section) #Note that cv2.imshow() needs an numpy array in it's argument
                                            #So if we don't convert it into a numpy array, it will throw errors
    cv2.imshow("Video Frame",frame)  
    cv2.imshow("Face Seciton", face_section)
    
    key_pressed = cv2.waitKey(1) & (0xFF)
    if key_pressed == ord('q'):
        break
        
#Convert our face_data into a numpy array
face_data = np.asarray(face_data)
face_data = face_data.reshape((face_data.shape[0],-1))
print(face_data.shape)

#Save this data into file system
np.save(dataset_path+file_name+'.npy',face_data)
print("Data Succesfully saved at "+dataset_path+file_name+'.npy')
    
cap.release()
cv2.destroyAllWindows()

Enter name of the person: Sunil
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
(35, 30000)
Data Succesfully saved at ./Face_Data_Store/Sunil.npy


In [2]:

#In order to load the saved numpy array and see the saved file
X = np.load("./Face_Data_Store/Sunil.npy")

In [3]:
print(X)

[[ 70  91 109 ... 100 115 121]
 [ 60  90 108 ...  96 110 127]
 [ 67  98 113 ... 101 116 128]
 ...
 [109 109 109 ... 141 132 131]
 [106 106 106 ... 146 140 134]
 [113 113 113 ... 126  95  90]]
